# Advanced Searching

---
---

The most basic searchs can be done using `db.view()`.

<u>Examples</u>:
* `db.view('uid')`                                              shows just that object (can be group, collection, etc.)
* `db.view(Collection)`                                         shows all collections from all groups you are apart of
* `db.view(Inventory)`                                          shows all inventories from all groups you are apart of
* `db.view(Experiment)`                                         shows all experiments from all groups and collections
* `db.view(Data)`                                               shows all data from all groups and collections
* `db.view(Group, {"scope": "all"})`                          shows all groups in database (default is first 50 results)
* `db.view(Material, {"scope": "all"}, num_results =100)`     shows all materials in database (changing `num_results` to get more results, but don't increase it too much <1000)

A power statagy when searching with `db.view()` is to start with known starting point, like a group or collection node, traverse down the graph to find the node you are looking for.

## <u>Advanced Searching</u>

Advanced searchs consists of search such as:
* show all materials with a molecular wieght between 100 g/mol and 500 g/mol
* show all materials with a glass transition temperature(t_g) above 150 oC
* show all materials with the word 'acrylic' in the name
* show all process with styrene as a monomer and a conversion over 95%

At the monoment, advanced searches have not been implemented in a user friendly way. It currently relies on MongoDB searching syntax and requires detailed knowledge of where the data you are looking for is. This is hoped to be improved in the future.

Nevertheless, here are a few examples and some important information to help out determined individuals wanting to these types of searches.
* [MongoDB search syntax](https://docs.mongodb.com/manual/reference/operator/query/)   
* [MongoDB examples](https://docs.mongodb.com/manual/tutorial/query-documents/)
* All quantities are converted into a database default units and the units is dropped prior to being saved into the database (just the value is saved). Thus, to search for properties that are numerical you are looking to search by the value in database units. You can find the database units for properties and conditions with `Prop.key_table()` or `Cond.key_table()`. 
* To know where to search it may be useful to look at exported documents of the same type. See the exporting tutorial with instructions on how to export.
* Also, remember that the first 50 results are shown by defualt. 

The first thing to do when doing an a search is of course login to the database.

In [1]:
from cript import *

db_username = "Olsen_lab"
db_password = "Brad_Olsen"
db_project = "cript_testing"
db_database = "test"
user = "johndoe@cript.edu"
db = CriptDB(db_username, db_password, db_project, db_database, user)

Connection to database 'test' successful.
Login as 'John Doe' was successful.


<u>**Example 1**</u>

Our first example will find all materials with a molecular weight less than 100 g/mol.  by defualt.

To explain how this quary was crafted:
* we know we are looking for a `Material` as that so that the first part. 
* we want to search the whole database so that's `"scope": "all"`
* we want to search for a property and we want to match on a specfic property `"mw" key": {"prop": {"$elemMatch": {"key": "mw"}}}`
* we want the value of that property to be less than ($lt) 100 g/mol (g/mol is the default database unit) ` "value": {"$lt": 100}`


In [2]:
result = db.view(Material, {"scope": "all", "key": {"prop": {"$elemMatch": {"key": "molar_mass", "value": {"$lt": 100}}}}})


number  name                          uid                           
------------------------------------------------------------
0       water                         614a3187568f442e6d75b4da      
1       nitrogen                      614a3188568f442e6d75b4dc      
2       argon                         614a3188568f442e6d75b4dd      
3       toluene                       614a3188568f442e6d75b4df      
4       tetrahydrofuran               614a3188568f442e6d75b4e0      
5       1-butanol                     614a3188568f442e6d75b4e1      
6       methanol                      614a3188568f442e6d75b4e2      
7       cyclohexane                   614a3189568f442e6d75b4e3      
8       sec-butyllithium              614a3189568f442e6d75b4e4      



<u>**Example 2**</u>

Our second example will find all materials with a glass transition temperature greater than 150 C. The result will only return the first 50 or less (if there is less than 50 materials that fit that quary) by defualt.

To explain how this quary was crafted:
* we know we are looking for a `Material` as that so that the first part. 
* we want to search the whole database so that's `"scope": "all"`
* we want to search for a property and we want to match on a specfic property `"t_g" key": {"prop": {"$elemMatch": {"key": "t_g"}}}`
* we want the value of that property to be less than ($gt) 150 C (C is the default database unit) ` "value": {"$gt": 150}`

In [3]:
result = db.view(Material, {"scope": "all", "key": {"prop": {"names":  {"key": "t_g", "value": {"$gt": 150}}}}})


number  name                          uid                           
------------------------------------------------------------
No results.


<u>**Example 3**</u>

Our third example will find all materials with styrene in the name. 


To explain how this quary was crafted:
* we know we are looking for a `Material` as that so that the first part. 
* we want to search the whole database so that's `"scope": "all"`
* we want to search for 'sec' in any of the names of a Material, in which names are found in the 'iden' section. `"key": {"iden": {"$elemMatch": {"names": }}}` <br>
    * The `"$elemMatch"` is because `iden` is a list.
* we will use regular expression (regex) to find the text styrene anywhere in a name `{'$regex': "sec"}

In [11]:
result = db.view(Material, {"scope": "all", "key": {"iden": {"$elemMatch": {"names": {'$regex': "sec"}}}}})


number  name                          uid                           
------------------------------------------------------------
0       sec-butyllithium              614a3189568f442e6d75b4e4      
1       SecBuLi solution 1.4M cHe     614a3189568f442e6d75b4e5      



To search for materials that have the letters "sec" in "name" and with a "mat_id" = 1.

In [9]:
result = db.view(Material, {"scope": "all", "key": {"iden": {"$elemMatch": {"name": {'$regex': "sec"}, "mat_id": 1}}}})


number  name                          uid                           
------------------------------------------------------------
0       sec-butyllithium              614a3189568f442e6d75b4e4      



<u>**Example 4**</u>

Our fourth example will find all materials have "s" in name **and** a molar mass greater than 100 g/mol. 


In [20]:
result_with_s = db.view(Material, {"scope": "all", "key": {"iden": {"$elemMatch": {"names": {'$regex': "s"}}}}})
results_with_mm_100 = db.view(Material, {"scope": "all", "key": {"prop": {"$elemMatch": {"key": "molar_mass", "value": {"$gt": 100}}}}})
result_with_both = db.view(Material, {"scope": "all", 
                                      "key": { 
                                          "$and": [
                                              {"iden": {"$elemMatch": {"names": {'$regex': "s"}}}}, 
                                              {"prop": {"$elemMatch": {"key": "molar_mass", "value": {"$gt": 100}}}}
                                          ]}})


number  name                          uid                           
------------------------------------------------------------
0       styrene                       614a3188568f442e6d75b4de      
1       sec-butyllithium              614a3189568f442e6d75b4e4      
2       SecBuLi solution 1.4M cHe     614a3189568f442e6d75b4e5      
3       polystyrene                   614a318a568f442e6d75b4f1      


number  name                          uid                           
------------------------------------------------------------
0       deuterated chloroform         614a3188568f442e6d75b4db      
1       styrene                       614a3188568f442e6d75b4de      


number  name                          uid                           
------------------------------------------------------------
0       styrene                       614a3188568f442e6d75b4de      

